<a href="https://colab.research.google.com/github/nezuk00/vis4/blob/main/lab_california_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Лабораторная работа: Классификация на основе датасета California Housing

В этой лабораторной работе используется датасет из **задания 1** (California Housing). Исходно задача была регрессионной – предсказание стоимости дома `median_house_value`. Здесь мы преобразуем её в задачу **классификации**, разбив целевой признак на интервалы, и исследуем влияние дисбаланса классов и методов балансировки на качество модели.

**План:**
1. Загрузить и подготовить данные: переименовать столбцы, создать категориальный признак `ocean_proximity` и разделить стоимость на интервалы.
2. Построить модель `DecisionTreeClassifier` на исходных данных, оценить точность (accuracy).
3. Искусственно создать дисбаланс классов, уменьшив один из классов до 10 % от максимального, обучить новую модель и оценить точность.
4. Применить методы балансировки (Random Oversampling, SMOTE, ADASYN, Tomek Links), для каждой получить модель и оценить точность.
5. Сравнить результаты и сделать выводы.


In [2]:

# Установим библиотеку imbalanced-learn (раскомментируйте при первом запуске)
!pip install -q imbalanced-learn


In [3]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Библиотеки для балансировки
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import TomekLinks

import warnings
warnings.filterwarnings('ignore')


In [21]:
# Загрузка датасета. Можно использовать функцию sklearn или локальный CSV-файл.
# Попробуем загрузить через sklearn; если нет интернета, загрузите локальный файл 'housing.csv'.
!pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import numpy as np

# Set the path to the file you'd like to load
file_path = "housing.csv"

# Load the latest version of California Housing Prices
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "camnugent/california-housing-prices",
  file_path,
)
# Переименуем столбцы как в предыдущей лабораторной
rename_dict = {
    'MedInc': 'median_income',
    'HouseAge': 'housing_median_age',
    'AveRooms': 'average_rooms',
    'AveBedrms': 'average_bedrooms',
    'Population': 'population',
    'AveOccup': 'households',
    'Latitude': 'latitude',
    'Longitude': 'longitude',
    'MedHouseVal': 'median_house_value'
}
df.rename(columns=rename_dict, inplace=True)
# Переведём стоимость в доллары
df['median_house_value'] *= 100000
# Создадим признак близости к океану
df['ocean_proximity'] = np.where(df['latitude'] >= 35, 'NEAR BAY', 'INLAND')


# Разобьём стоимость на 3 категории по квартилям
# qcut автоматически делает интервалы примерно одинаковой мощности
n_bins = 3
df['price_category'] = pd.qcut(df['median_house_value'], q=n_bins, labels=['Low','Mid','High'])

# Удалим исходный числовой столбец
X = df.drop(['median_house_value', 'price_category'], axis=1)
y = df['price_category']

print('Распределение классов:')
print(y.value_counts())


Using Colab cache for faster access to the 'california-housing-prices' dataset.
Распределение классов:
price_category
Low     6884
High    6880
Mid     6876
Name: count, dtype: int64


In [6]:

# Определяем числовые и категориальные признаки
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = ['ocean_proximity']

# Создаём колонковый трансформер: нормализация числовых и OneHotEncoding категорий
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),  # для дерева масштабирование не требуется
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)

# Делим данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Собираем пайплайн: предобработка + модель
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

# Обучение
model.fit(X_train, y_train)

# Оценка на тестовой выборке
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f'Accuracy исходной модели: {acc:.4f}')
print('\nМатрица ошибок:')
print(confusion_matrix(y_test, y_pred))
print('\nОтчет классификации:')
print(classification_report(y_test, y_pred))


Accuracy исходной модели: 0.7413

Матрица ошибок:
[[1587   59  418]
 [  79 1642  344]
 [ 368  334 1361]]

Отчет классификации:
              precision    recall  f1-score   support

        High       0.78      0.77      0.77      2064
         Low       0.81      0.80      0.80      2065
         Mid       0.64      0.66      0.65      2063

    accuracy                           0.74      6192
   macro avg       0.74      0.74      0.74      6192
weighted avg       0.74      0.74      0.74      6192



In [7]:

# Создаем искусственный дисбаланс: выберем класс с наибольшим количеством объектов
class_counts = y_train.value_counts()
max_class = class_counts.idxmax()
min_class_count = int(class_counts.max() * 0.10)  # 10% от самого большого класса

# Выберем индексы объектов этого класса в обучающей выборке и оставим только min_class_count
idx_max_class = y_train[y_train == max_class].index
idx_to_keep = idx_max_class[:min_class_count]
# Инверсия: сохраняем остальные классы полностью
idx_other_classes = y_train[y_train != max_class].index

# Новый обучающий набор
X_train_imb = pd.concat([X_train.loc[idx_to_keep], X_train.loc[idx_other_classes]], axis=0)
y_train_imb = pd.concat([y_train.loc[idx_to_keep], y_train.loc[idx_other_classes]], axis=0)

print('Распределение классов в несбалансированной обучающей выборке:')
print(y_train_imb.value_counts())

# Обучаем новую модель на несбалансированных данных
model_imb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(random_state=42))
])

model_imb.fit(X_train_imb, y_train_imb)
y_pred_imb = model_imb.predict(X_test)
acc_imb = accuracy_score(y_test, y_pred_imb)
print(f'Accuracy модели на дисбалансированных данных: {acc_imb:.4f}')


Распределение классов в несбалансированной обучающей выборке:
price_category
High    4816
Mid     4813
Low      481
Name: count, dtype: int64
Accuracy модели на дисбалансированных данных: 0.6610


In [13]:
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import TomekLinks
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

samplers = {
    'Без балансировки': None,
    'Random Oversampling': RandomOverSampler(random_state=42),
    'SMOTE': SMOTE(random_state=42),
    # Генерируем новые объекты только для самого маленького класса
    'ADASYN': ADASYN(random_state=42, sampling_strategy='minority'),
    'Tomek Links': TomekLinks()
}

results = {}

# трансформируем наши несбалансированные данные
X_train_imb_transformed = preprocessor.transform(X_train_imb)
X_test_transformed = preprocessor.transform(X_test)

# превращаем разреженную матрицу в плотный массив (важно для SMOTE/ADASYN)
if hasattr(X_train_imb_transformed, 'toarray'):
    X_train_imb_transformed = X_train_imb_transformed.toarray()
    X_test_transformed = X_test_transformed.toarray()

for name, sampler in samplers.items():
    if sampler is None:
        X_res, y_res = X_train_imb_transformed, y_train_imb
    else:
        X_res, y_res = sampler.fit_resample(X_train_imb_transformed, y_train_imb)
    clf = DecisionTreeClassifier(random_state=42)
    clf.fit(X_res, y_res)
    y_pred_bal = clf.predict(X_test_transformed)
    results[name] = accuracy_score(y_test, y_pred_bal)
    print(f'{name}: accuracy = {results[name]:.4f}')


Без балансировки: accuracy = 0.6610
Random Oversampling: accuracy = 0.6118
SMOTE: accuracy = 0.6959
ADASYN: accuracy = 0.6883
Tomek Links: accuracy = 0.6526



## Выводы

В ходе выполнения задания была проведена классификация на основе датасета California Housing, преобразованного из задачи регрессии. Были сформированы три класса (Low, Mid, High) по стоимости жилья.

- Базовая модель дерева решений на исходных данных показала некоторую точность (см. выводы выше).
- Искусственное создание дисбаланса, уменьшив самый распространённый класс до 10 %, привело к снижению качества классификации. Это ожидаемо, поскольку дерево решений склонно уделять больше внимания большинству классов.
- Применение методов балансировки:
  - **Random Oversampling** воспроизводит экземпляры миноритарных классов, повышая их долю.
  - **SMOTE** и **ADASYN** синтетически создают новые точки в миноритарных классах, тем самым улучшая обучающую выборку.
  - **Tomek Links** удаляет приграничные объекты, стремясь уменьшить перекрытие классов.

При сравнении accuracy видно, что методы синтетического увеличения (SMOTE, ADASYN) обычно дают лучший или сопоставимый результат по сравнению с простым копированием. Метод Tomek Links, являющийся скорее методом очистки, может давать неоднозначный эффект. Общий вывод: для задач с дисбалансом классов важно использовать подходящие методы балансировки, поскольку они способны значительно повысить качество классификации.
